In [60]:
import polars as pl
import matplotlib.pyplot as plt

In [61]:
df = pl.read_csv('./log-3.csv')

In [62]:
df.head()

chosen_dag,num_clients,num_samples,comparison_category,same_p_val,missed_tests,total_features,features_per_client,correctness_alpha_0.01,correctness_alpha_0.05,correctness_alpha_0.1
str,i64,i64,str,f64,f64,i64,i64,f64,f64,f64
"""chain4""",1,1000,"""ground_truth""",0.666667,0.75,4,3,1.0,0.833333,1.0
"""chain4""",3,1000,"""ground_truth""",0.363636,0.541667,4,3,0.727273,0.727273,0.863636
"""chain4""",5,1000,"""ground_truth""",0.333333,0.25,4,3,0.944444,0.888889,0.972222
"""chain4""",10,1000,"""ground_truth""",0.333333,0.25,4,3,0.805556,0.916667,0.833333
"""chain4""",1,1000,"""real""",0.833333,0.75,4,3,0.833333,1.0,1.0


In [63]:
df.plot.box(y='correctness_alpha_0.01', by='num_clients')

:BoxWhisker   [num_clients]   (correctness_alpha_0.01)

In [64]:
df_comp2real = df.filter(pl.col('comparison_category') == 'real')
df_comp2citest = df.filter(pl.col('comparison_category') == 'ground_truth')

In [65]:
df_comp2real.plot.box(y='correctness_alpha_0.01', by='num_clients')

:BoxWhisker   [num_clients]   (correctness_alpha_0.01)

In [66]:
df_incomplete_features = df_comp2real.filter(pl.col('total_features') != pl.col('features_per_client'))

In [67]:
df_incomplete_features.plot.box(y='correctness_alpha_0.01', by='num_clients')

:BoxWhisker   [num_clients]   (correctness_alpha_0.01)

In [68]:
df_incomplete_features.plot.box(y='missed_tests', by='num_clients')

:BoxWhisker   [num_clients]   (missed_tests)

In [46]:
df_incomplete_features

chosen_dag,num_clients,num_samples,comparison_category,same_p_val,missed_tests,total_features,features_per_client,correctness_alpha_0.01,correctness_alpha_0.05,correctness_alpha_0.1
str,i64,i64,str,f64,f64,i64,i64,f64,f64,f64
"""chain4""",1,1000,"""real""",0.0,2.5,3,2,1.0,1.0,1.0
"""chain4""",3,1000,"""real""",1.0,1.0,3,2,1.0,1.0,1.0
"""chain4""",5,1000,"""real""",0.5,1.0,3,2,0.5,0.5,1.0
"""chain4""",10,1000,"""real""",0.666667,0.5,3,2,1.0,1.0,1.0
"""chain4""",1,1000,"""real""",1.0,2.5,3,2,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…
"""chain4""",10,1000,"""real""",0.666667,0.5,3,2,0.666667,1.0,1.0
"""chain4""",1,1000,"""real""",1.0,2.5,3,2,1.0,1.0,1.0
"""chain4""",3,1000,"""real""",0.333333,0.5,3,2,1.0,1.0,1.0


In [50]:
df_comp2citest.plot.box(y='correctness_alpha_0.01', by='num_clients', ylim=(-0.1,1.1))

:BoxWhisker   [num_clients]   (correctness_alpha_0.01)

In [69]:
def filter_to_complete_featuresets_per_client(df):
    return df.filter(pl.col('total_features') == pl.col('features_per_client'))

def filter_to_incomplete_featuresets_per_client(df):
    return df.filter(pl.col('total_features') != pl.col('features_per_client'))

def filter_to_citest_comparison(df):
    return df.filter(pl.col('comparison_category') == 'ground_truth')

def filter_to_real_comparison(df):
    return df.filter(pl.col('comparison_category') == 'real')

In [70]:
df2 = df
df2 = filter_to_complete_featuresets_per_client(df2)
df2 = filter_to_real_comparison(df2)
df2.head()

chosen_dag,num_clients,num_samples,comparison_category,same_p_val,missed_tests,total_features,features_per_client,correctness_alpha_0.01,correctness_alpha_0.05,correctness_alpha_0.1
str,i64,i64,str,f64,f64,i64,i64,f64,f64,f64
"""chain4""",1,1000,"""real""",0.708333,0.0,4,4,0.875,0.875,0.875
"""chain4""",3,1000,"""real""",0.708333,0.0,4,4,0.875,0.875,0.875
"""chain4""",5,1000,"""real""",0.75,0.0,4,4,0.875,0.875,0.875
"""chain4""",10,1000,"""real""",0.75,0.0,4,4,0.875,0.875,0.875
"""chain4""",1,1000,"""real""",0.708333,0.0,4,4,0.791667,0.875,0.875


In [86]:
df2.plot.box(y='correctness_alpha_0.01', by=['num_clients', 'num_samples'], c='num_clients', cmap='Pastel2')

:BoxWhisker   [num_clients,num_samples]   (correctness_alpha_0.01)